# Day 2: Red-Nosed Reports

Fortunately, the first location The Historians want to search isn't a long walk from the Chief Historian's office.

While the [Red-Nosed Reindeer nuclear fusion/fission plant](https://adventofcode.com/2015/day/19) appears to contain no sign of the Chief Historian, the engineers there run up to you as soon as they see you. Apparently, they _still_ talk about the time Rudolph was saved through molecular synthesis from a single electron.

They're quick to add that - since you're already here - they'd really appreciate your help analyzing some unusual data from the Red-Nosed reactor. You turn to check if The Historians are waiting for you, but they seem to have already divided into groups that are currently searching every corner of the facility. You offer to help with the unusual data.

The unusual data (your puzzle input) consists of many _reports_, one report per line. Each report is a list of numbers called _levels_ that are separated by spaces. For example:

```
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
```

This example data contains six reports each containing five levels.

The engineers are trying to figure out which reports are _safe_. The Red-Nosed reactor safety systems can only tolerate levels that are either gradually increasing or gradually decreasing. So, a report only counts as safe if both of the following are true:

- The levels are either _all increasing_ or _all decreasing_.
- Any two adjacent levels differ by _at least one_ and _at most three_.

In the example above, the reports can be found safe or unsafe by checking those rules:

- `7 6 4 2 1`: _Safe_ because the levels are all decreasing by 1 or 2.
- `1 2 7 8 9`: _Unsafe_ because `2 7` is an increase of 5.
- `9 7 6 2 1`: _Unsafe_ because `6 2` is a decrease of 4.
- `1 3 2 4 5`: _Unsafe_ because `1 3` is increasing but `3 2` is decreasing.
- `8 6 4 4 1`: _Unsafe_ because `4 4` is neither an increase or a decrease.
- `1 3 6 7 9`: _Safe_ because the levels are all increasing by 1, 2, or 3.

So, in this example, `_2_` reports are _safe_.

Analyze the unusual data from the engineers. _How many reports are safe?_

In [20]:
$sample_data = @'
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
'@ -split "`r`n"

$input_data = Get-Content .\data.txt

In [21]:
# For part 2, but putting this & the subsequent cell here so 'run all' in the notebook still works gracefully.
# This was actually my OG function, doing it 'efficiently', but it only took 0.4s to get the wrong result.
# So YOLO, it's OK to brute force it on day 2, right...?

function problemDampener {
    [CmdletBinding()] Param(
        [int[]] $reports,
        [int] $i
    )
    # either $i or $i-1 is bad. Try dropping $i-1 first.
    if($i -eq 1) { #edge case where we're just at the start
        $newline = $reports[$i..($reports.Count-1)]
    } else {
        $newline = $reports[0..($i-2)] + $reports[$i..($reports.Count-1)]
    }
    Write-Debug "Problem dampener active for line $($reports -join ' '), trying $($newline -join ' ')"
    if(!(testReport $newline)) {
        #last chance bus! test if $i was bad.
        if($i -eq ($reports.Count-1)) {
            $newline = $reports[0..($i-1)]
        } else {
            $newline = $reports[0..($i-1)] + $reports[($i+1)..($reports.Count-1)]
        }
        Write-Debug "Problem dampener active (last chance!) for line $($reports -join ' '), trying $($newline -join ' ')"
        return (testReport $newline)
    } else {
        return $true
    }
}

In [22]:
function problemDampener {
    [CmdletBinding()] Param(
        [int[]] $reports,
        [int] $i
    )

    for($j=0;$j -lt ($reports.Count);$j++) {
        if($j -eq 0) { #edge case where we're just at the start
            $newline = $reports[1..($reports.Count-1)]
        } elseif($j -eq ($reports.Count-1)) { # or at the end
            $newline = $reports[0..($j-1)]
        } else {
            $newline = $reports[0..($j-1)] + $reports[($j+1)..($reports.Count-1)]
        }
        Write-Verbose "Problem dampener active for line $($reports -join ' '), trying $($newline -join ' ')"
        if(testReport $newline -Debug:$false) {
            Write-Debug "Problem dampener successfully recovered line $($reports -join ' '), using $($newline -join ' ')"
           return $true 
        }
    }
    Write-Debug "Problem dampener couldn't recover line $($reports -join ' ')"
    return $false
}

In [23]:
function dostuff {
    [CmdletBinding()] Param(
        [Parameter(Mandatory=$false)]
        [switch] $sample,
        [Parameter(Mandatory=$false)]
        [switch] $part2
    )

    if($sample) {
        $data = $sample_data
    } else {
        $data = $input_data
    }

    $totalSafeReports = 0

    foreach($line in $data) {
        $reports = [int[]] ($line -split ' ')

        if(!$part2) {
            if(!(testReport $reports)) {
                continue
            }
        } else {
            if(!(testReport $reports -allowDampening)) {
                continue
            }

        }
        $totalSafeReports++
    }
    Write-Output "Of $($data.Count) reports, there are $totalSafeReports that are safe."
}

function testReport {
    [CmdletBinding()] Param(
        [int[]] $reports,
        [switch] $allowDampening
    )
    $increasing = $false
    if($reports[0] -lt $reports[1]) {
        $increasing = $true
    }
    for($i=1; $i -lt $reports.Count; $i++) {
        if([Math]::Abs($reports[$i-1] - $reports[$i]) -notin @(1,2,3)) {
            Write-Debug "Line $($reports -join ' ') is unsafe because it has reports ($($reports[$i-1]), $($reports[$i])) with a difference ($([Math]::Abs($reports[$i-1] - $reports[$i]))) <3 or >1."
            if($allowDampening) {
                if(problemDampener $reports $i) {
                    return $true
                }
            }
            return $false
        }
        if($increasing) {
            if($reports[$i-1] -gt $reports[$i]) {
                Write-Debug "Line $($reports -join ' ') is unsafe because it's increasing, but $($reports[$i-1]) > $($reports[$i])"
                if($allowDampening) {
                    if(problemDampener $reports $i) {
                        return $true
                    }
                }
                return $false
            }
        } else {
            if($reports[$i-1] -lt $reports[$i]) {
                Write-Debug "Line $($reports -join ' ') is unsafe because it's decreasing, but $($reports[$i-1]) < $($reports[$i])"
                if($allowDampening) {
                    if(problemDampener $reports $i) {
                        return $true
                    }
                }
                return $false
            }

        }
    }
    return $true
}


dostuff -sample -Debug
dostuff
dostuff -sample -Debug -part2
dostuff -part2

DEBUG: Line 1 2 7 8 9 is unsafe because it has reports (2, 7) with a difference (5) <3 or >1.
DEBUG: Line 9 7 6 2 1 is unsafe because it has reports (6, 2) with a difference (4) <3 or >1.
DEBUG: Line 1 3 2 4 5 is unsafe because it's increasing, but 3 > 2
DEBUG: Line 8 6 4 4 1 is unsafe because it has reports (4, 4) with a difference (0) <3 or >1.
Of 6 reports, there are 2 that are safe.
Of 1000 reports, there are 224 that are safe.
DEBUG: Line 1 2 7 8 9 is unsafe because it has reports (2, 7) with a difference (5) <3 or >1.
DEBUG: Problem dampener couldn't recover line 1 2 7 8 9
DEBUG: Line 9 7 6 2 1 is unsafe because it has reports (6, 2) with a difference (4) <3 or >1.
DEBUG: Problem dampener couldn't recover line 9 7 6 2 1
DEBUG: Line 1 3 2 4 5 is unsafe because it's increasing, but 3 > 2
DEBUG: Problem dampener successfully recovered line 1 3 2 4 5, using 1 2 4 5
DEBUG: Line 8 6 4 4 1 is unsafe because it has reports (4, 4) with a difference (0) <3 or >1.
DEBUG: Problem dampener su

# Part Two

The engineers are surprised by the low number of safe reports until they realize they forgot to tell you about the Problem Dampener.

The Problem Dampener is a reactor-mounted module that lets the reactor safety systems _tolerate a single bad level_ in what would otherwise be a safe report. It's like the bad level never happened!

Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe.

More of the above example's reports are now safe:

- `7 6 4 2 1`: _Safe_ without removing any level.
- `1 2 7 8 9`: _Unsafe_ regardless of which level is removed.
- `9 7 6 2 1`: _Unsafe_ regardless of which level is removed.
- `1 _3_ 2 4 5`: _Safe_ by removing the second level, `3`.
- `8 6 _4_ 4 1`: _Safe_ by removing the third level, `4`.
- `1 3 6 7 9`: _Safe_ without removing any level.

Thanks to the Problem Dampener, `_4_` reports are actually _safe_!

Update your analysis by handling situations where the Problem Dampener can remove a single level from unsafe reports. _How many reports are now safe?_